In [ ]:
import urllib, re, json
from urllib.parse import urlparse
import urllib.request
import pandas as pd
import numpy as np

#from pyquery import PyQuery
from bs4 import BeautifulSoup
try:
    from http.cookiejar import CookieJar
except ImportError:
    from cookielib import CookieJar

<h6>Web Scraping con python 3</h6>

In [ ]:
login_param = {'txtUsuario':'2123065423',
         'txtContrasena':'Oswaldo948',
         'btnAccesar':'Ingresar'}
cj = CookieJar()
opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
resp1 = opener.open('https://www.bolsadetrabajo.uam.mx/sesion.php', urllib.parse.urlencode(login_param).encode("utf-8"))

In [ ]:
page_param = {'txtClave':'Oswaldo948',
             'txtUsuario':'2123065423',
             'x':'79',
             'y':'24'}
resp2 = opener.open('https://www.bolsadetrabajo.uam.mx/srcBuscaEmp.php',urllib.parse.urlencode(page_param).encode("utf-8"))
html_text = resp2.read()

<h4>Funciones</h4>

In [ ]:
def cleanStr(string):
    return string.replace('\t',' ').strip(' \t\n\r').replace('\n',' ').replace('\r',' ').replace('•',' ')
def quitarLatin(string):
    string.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u').replace('ñ','n')
def isfloat(x):
    return re.match("^\d+(\.\d+)?", x) is not None

<h3>Carga de datos</h3>

In [ ]:
with open('vacantes.json') as f:    
    data = json.load(f)
#vac = pd.DataFrame.from_dict(data, orient='columns')
vac = pd.DataFrame.from_dict(data, orient='index')

<h3>Renombrar columnas</h3>

In [ ]:
vac.columns = vac.columns.str.lower().str.strip().str.replace(' ','_')
vac.columns = vac.columns.str.replace('á','a').str.replace('é','e').str.replace('í','i').str.replace('ó','o').str.replace('ú','u').str.replace('ñ','n')
vac.rename(columns={'disponibilidad_para_radicar_en_provincia': 'radicar_provincia', 
                    'disponibilidad_para_rolar_turnos': 'rolar_turnos',
                   'fecha_de_registro_de_la_vacante':'fecha_registro',
                   'tipo_de_plaza':'tipo_plaza',
                    'horario_de_salida':'horario_salida'}, inplace=True)
vac.columns

<h3>Limpiar datos</h3>

In [ ]:
vac.experiencia = vac.experiencia.apply(cleanStr)

#vac.sueldo_maximo = vac.sueldo_maximo.str.replace('$','').str.replace(',','')
vac.sueldo_maximo = vac.sueldo_maximo.str.replace('$','').str.replace(',','')
vac.sueldo_maximo[vac.sueldo_maximo.apply(isfloat)==False]=np.nan
vac.sueldo_maximo = vac.sueldo_maximo.astype(float)

vac.sueldo_minimo = vac.sueldo_minimo.str.replace('$','').str.replace(',','').str.replace('|','')
vac.sueldo_minimo = vac.sueldo_minimo.dropna().astype(float)

#vac.edad_minima = vac.edad_minima.str.replace('$','').str.replace(',','')
vac.edad_minima = vac.edad_minima.dropna().astype(int)
vac.edad_maxima = vac.edad_maxima.astype(int)

vac.fecha_registro = pd.to_datetime(vac.fecha_registro)

vac.dtypes

<h2>Mejor sueldo</h2>

In [ ]:
vac.groupby('carrera')['sueldo_maximo'].mean().sort_values(ascending=False)

In [ ]:
vac.groupby('carrera')['sueldo_minimo'].mean().sort_values(ascending=False)

In [ ]:
tab = pd.concat([vac.carrera,(vac.sueldo_maximo+vac.sueldo_minimo)/2],axis=1).rename(columns={0:'sueldo'})
tab.groupby('carrera')['sueldo'].mean().sort_values(ascending=False)

<h2>Oportunidades de trabajo</h2>

In [ ]:
vac.groupby('carrera').size().sort_values(ascending=False)

<h2>Oportunidades de trabajo dependiendo el sexo</h2>

In [ ]:
vac.groupby('sexo').size().sort_values(ascending=False)

In [ ]:
re.sub(' +',' ','The     quick brown    fox')

<h1>Graficas</h1>

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
oport = vac.groupby('carrera').size().sort_values(ascending=False)
labels = tuple(oport.keys())
sizes = np.array(oport,dtype=pd.Series)*100/np.sum(np.array(oport,dtype=pd.Series))
colors = ['yellowgreen', 'mediumpurple', 'lightskyblue', 'lightcoral','yellowgreen', 'mediumpurple', 'lightskyblue', 'lightcoral','yellowgreen', 'mediumpurple', 'lightskyblue', 'lightcoral','yellowgreen', 'mediumpurple'] 
explode = (0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0,0, 0)    # proportion with which to offset each wedge
plt.axis('equal')
plt.pie(sizes,              # data
        explode=explode,    # offset parameters 
        labels=labels,      # slice labels
        colors=colors,      # array of colours
        autopct='%1.1f%%',  # print the values inside the wedges
        shadow=True,        # enable shadow
        startangle=70       # starting angle
        )